# Pre-processing
\
The raw data are point clouds generated by MLS (mobile LiDAR). The very first pre-processing step involves processing by the SLAM algorithm (more specifically, via the application provided by GeoSLAM), and unzip files.

Then, the next part consists in classifying ground points, generating a DTM, and extracting vegetation points in a slice parallel to DTM. This step is carried out using the opensource software Computree, and is detailed in the file computree_steps.xsct2 (it is ready to use, only the input files need to be selected and the output folder specified, parameters can also be viewed and modified).

Note: depending on the device you are using, it may be necessary to tile the point cloud before processing with Computree. In this case, make sure you convert to point format 7 and merge the output tile files before moving on. I recommend using CloudCompare for tiling and for converting .laz to .las.

*Output folder: 'computree_outputs'*

# Clustering
\
A shape (branch, trunk, etc.) is characterised by a high point density. To eliminate noise such as foliage, point clouds are clustered using a DBSCAN (density-based spatial clustering of applications with noise) algorithm. The resulting clusters are then clustered more finely using the HDBSCAN method, and clusters are filtered in order to eliminate high branches and shapes that are short in length.
In case, other filters are available, based on a minimum distance from plot centre or a minimum number of cluster points.

The clustering results are exported as a point cloud (.las file), and as individual cluster images for classification.

*Input folder: 'computree_outputs'*  
*Output folders: 'clusters_las' (.las files), 'clusters_img' (.png files)*

In [ ]:
# Loading packages and modules
import os
import glob
import laspy
from clustering import ClEngine, Cluster

las_files_path = 'computree_outputs' # location of preprocessed files
clusters_las_path = 'clusters_las' # location of clustered .las files
clusters_img_path = 'clusters_img' # location of cluster images

# Cluster image params
figsize = (4,4)
dpi = 75

# Listing all .las files to cluster
las_files = glob.glob(las_files_path + '/*.las')

# Listing already clustered files
clustered = glob.glob(clusters_las_path + '/*.las')
clustered_names = [os.path.splitext(os.path.basename(file))[0] for file in clustered]

In [ ]:
for file in las_files:
    
    # Checking if the file is not already clustered
    if os.path.splitext(os.path.basename(file))[0]+'_cl' not in clustered_names:
        
        cl = ClEngine(file)
        
        # Cluster .las file points with DBSCAN
        cl.DBSCAN_clustering(eps=0.04, min_samples=50)
        cl.save_all_points(folder=clusters_las_path, suffix="dbscan") # backup of DBSCAN clusters
        
        # Cluster points finely using HDBSCAN algorithm (when there is too much vegetation points)
        cl.HDBSCAN_clustering(min_cluster_size=500, max_cluster_size=250000, min_samples=25)
        cl.save_all_points(folder=clusters_las_path, suffix="hdbscan") # backup of HDBSCAN clusters
        
        # Filter clusters given on a minimum number of points, distance from the plot centre, distance from the ground,
        # and minimum length. Set argument to None to ignore filter.
        cl.filtering(delta=0.75, min_dist=0.5)
        
        # Save final clustering results in a new .las file
        cl.save_all_points(folder=clusters_las_path, suffix="cl")
        
        # Save clustering results in .png files
        cl.save_individual_images(folder=clusters_img_path, figsize=figsize, dpi=dpi)

## NNCLR model building
\
The aim is not to spend our time manually classifying images, so we will be using a NNCLR model, adapted to a small labelled training set. The NNCLR model used here is based on the example of https://keras.io/examples/vision/nnclr/ (see link for more details).

A training dataset was created from manually classified images of 2 sample plots (07_04 and BM08), and unlabelled images
from 4 sample plots (BM01, BM02, BM03, BM04). Labelled images were divided into 2 classes: "deadwood" and "other". The folder architecture for training is as follows:


```
NNCLR_data
│
├── labelled
│   ├── deadwood
│   │   ├── image1.png
│   │   ├── image2.png
│   │   ├── ...
│   │
│   └── other
│       ├── image1.png
│       ├── image2.png
│       ├── ...
│
└── unlabelled
    └── unlabelled
        ├── image1.png
        ├── image2.png
        ├── ...
```

*Input/output folder: 'NNCLR_data', containing images for training*

Model hyperparameters, for information:

```
input_shape = (300, 300, 3)
shuffle_buffer = 5000   
temperature = 0.1
queue_size = 98304
```


In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import image_classification as imgclf

model_path = 'NNCLR_data' # path with labelled and unlabelled images for training
device = 'CPU:0' # device used to run tensorflow
batch_size = 32
num_epochs = 200 # max number of epochs, will stop before because early stopping is implemented
image_size = (300, 300) # in pixels

In [ ]:
# If GPU memory is insufficient, the CPU can be used (calculations will take longer)
with tf.device(device):
    
    # Initialise the model
    model = imgclf.Model(model_path=model_path, image_size=image_size, batch_size=batch_size,
                         num_epochs=num_epochs, encoder='custom')
    
    # Prepare training and validation datasets
    model.prepare_dataset()
    
    # Visualise augmentations
    # model.visualize_augmented_images(model._labelled_train_ds, num_images=10)
    
    # Pre-train NNCLR
    model.pretraining(patience=10) # keep weights such that val_p_loss is the lowest and has not increased in 10 epochs
    
    # Perform fine tuning, save finetuning model
    model.finetuning(patience=20, save=True)

## Image classification
\
*Input folders: 'NNCLR_data', 'clusters_las', 'clusters_img'*  
*Output folder: 'deadwood'*

In [ ]:
import os
import glob
import laspy
import numpy as np
import tensorflow as tf
import image_classification as imgclf
from clustering import ClEngine, Cluster, label_from

# Needed paths
model_path = 'NNCLR_data'
path_las = 'clusters_las'
path_img = 'clusters_img'
path_dw = 'deadwood'

device = 'CPU:0'

# Get already classified file names
classified_files = glob.glob(path_dw+'/*.las')
classified_names = [os.path.splitext(os.path.basename(file))[0] for file in classified_files]

In [ ]:
with tf.device(device):

    # Re-create the model
    model = imgclf.Model(model_path=model_path, load=True)

In [ ]:
with tf.device(device):

    # Browse of each study area folder
    for folder in glob.glob(path_img+'/*'):

        folder_name = os.path.splitext(os.path.basename(folder))[0]

        # Check if classification is not already done
        if folder_name + '_dw' not in classified_names:

            las_file = path_las + '/' + folder_name + '_cl.las'

            # Load clusters
            cl = ClEngine(las_file)
                        
            # List of cluster images
            images = glob.glob(folder+'/*.png')

            # Make a prediction with the model: each image classified as deadwood by the model, and
            # classified as "other" but with a score < threshold are kept in the final point cloud
            deadwood_images = model.prediction(images)

            # List of clusters (cluster labels) classified as deadwood
            deadwood_labels = [label_from(image) for image in deadwood_images]

            # Keep clusters classified as deadwood
            cl.keep_clusters(cluster_list=deadwood_labels)
            
            # Calculate cluster volumes
            cl.calculate_volumes(alpha=50)
            
            # Save clusters classified as deadwood
            cl.save_all_points(folder=path_dw, suffix='dw')
            
            # Save a csv file with all deadwood volume
            cl.save_volumes(folder=path_dw, suffix='volumes_dw')
            
            # Save inventory volumes
            cl.filtering(distance_from_centre=19)
            cl.save_volumes_csv(folder=path_dw, suffix='volumes_inv')